
## 1. 학습 데이터 전처리


* **결측치 처리 (Handling Missing Values):**
    * `TotalCharges` (총요금) 컬럼이 숫자형이 아닌 문자열(object) 타입이었으며, 일부 값은 `null`이 아닌 공백(' ')으로 존재했습니다.
    * `isnull()`로 감지되지 않는 이 공백 값들을 **바로 앞 행의 값으로 채우는 방식**(fillna)을 사용했습니다.
    * 이후 해당 컬럼을 숫자형(float)으로 변환했습니다.

* **범주형 데이터 인코딩 (Categorical Feature Encoding):**
    * 데이터셋의 텍스트 기반 범주형 피처(예: `gender`, `InternetService`, `Contract` 등)들은 `sklearn`의 **`LabelEncoder`**를 사용하여 모두 숫자형으로 일괄 변환했습니다.

* **불필요한 피처 제거 (Feature Dropping):**
    * 분석에 불필요한 `customerID` 컬럼을 제거했습니다.

---

## 2. 모델 적용

모델 학습 및 평가를 위해 다음과 같은 방법들이 사용되었습니다.

* **데이터 분할 (Data Splitting):**
    * 전체 데이터를 학습 데이터 80%, 테스트 데이터 20%의 비율로 분할 (`train_test_split`) 하였습니다.

* **적용된 모델 (Models Applied):**
    * 다음 4가지의 개별 머신러닝 모델을 적용했습니다.
        1.  **XGBoost Classifier** (`XGBClassifier`)
        2.  **LightGBM Classifier** (`LGBMClassifier`)
        3.  **Random Forest Classifier** (`RandomForestClassifier`)
        4.  **Decision Tree Classifier** (`DecisionTreeClassifier`)

* **모델 앙상블 (Ensemble):**
    * **스태킹(Stacking)** 기법을 사용하여 위 4가지 개별 모델을 결합했습니다.
    * `StackingClassifier`를 사용했으며, 최종 예측기(`final_estimator`)로는 `LightGBMClassifier`를 설정했습니다.

* **모델 평가 (Model Evaluation):**
    * **교차 검증 (Cross-Validation):** `RepeatedStratifiedKFold` (10-Fold, 3회 반복)를 사용하여 모델의 안정성과 일반화 성능을 측정했습니다.
    * **평가 지표:** `ROC_AUC` 점수를 핵심 지표로 사용했으며, 추가로 `Confusion Matrix` (혼동 행렬), `Classification Report` (정밀도, 재현율, F1 점수)를 통해 모델 성능을 다각도로 평가했습니다.


One-Hot Encoding (OHE) 대신 `LabelEncoder`가 사용된 주된 이유는 **사용한 머신러닝 모델이 트리(Tree) 기반 모델이기 때문**입니다.

이 노트북은 **XGBoost, LightGBM, Random Forest, Decision Tree** 등을 사용했습니다.

이러한 트리 기반 모델들은 데이터를 분기(split)하는 방식으로 작동합니다. `LabelEncoder`를 통해 'Month-to-month'가 0, 'One year'가 1, 'Two year'가 2로 변환되더라도, 모델은 이 값들을 수치적 크기(2가 1보다 2배 크다)로 해석하는 것이 아니라, **"값이 1 미만인가?"** 또는 **"값이 2인가?"**와 같은 분기 기준으로만 사용합니다.

따라서 트리 모델에게는 `LabelEncoder`만으로도 범주를 구분하는 데 충분하며, 굳이 OHE를 사용할 필요가 없습니다.

---

### OHE를 사용하지 않았을 때의 이점

1.  **차원(Feature) 수 유지:** OHE는 범주의 개수만큼 새로운 컬럼을 만듭니다. 이로 인해 데이터의 차원이 급격히 늘어날 수 있습니다(차원의 저주). `LabelEncoder`는 차원 수를 늘리지 않아 모델 학습이 더 빠르고 간결합니다.
2.  **간결성:** 모든 범주형 변수를 `LabelEncoder`로 일괄 처리하는 것이 코드상 더 간단합니다.

**반면에,** 만약 이 노트북이 **로지스틱 회귀(Logistic Regression)나 서포트 벡터 머신(SVM)**과 같은 선형 기반 모델을 사용했다면, `LabelEncoder`를 사용했을 때 문제가 될 수 있습니다. 선형 모델은 0, 1, 2라는 값 자체에 수치적인 가중치를 부여하며 'Two year'(2)가 'One year'(1)보다 2배의 영향력을 갖는다고 잘못 학습할 수 있기 때문입니다. 이런 경우에는 **반드시 One-Hot Encoding을 사용해야 합니다.**    